# Collection des documents (articles d'actualites)

In [1]:
import os
import requests
import json
from newspaper import Article

def fetch_articles(api_key, domains, save_path="corpus/articles_actualite", metadata_file="all_documents.json"):
    """
    Récupère des articles d'actualité à partir d'une API, extrait leur contenu textuel,
    et enregistre les articles et leurs métadonnées.

    Args:
        api_key (str): Clé API pour accéder à l'API des actualités.
        domains (list): Liste des catégories d'actualités à récupérer
                        (ex. ['science', 'sports', 'technology']).
        save_path (str): Répertoire pour sauvegarder les articles extraits.
        metadata_file (str): Fichier JSON pour sauvegarder les métadonnées des articles.
    """
    # Charger les métadonnées existantes ou initialiser une liste vide
    if os.path.exists(metadata_file):
        with open(metadata_file, "r", encoding="utf-8") as f:
            metadata = json.load(f)
    else:
        metadata = []

    article_id = len(metadata) + 1  # Définir l'ID du premier nouvel article

    base_url = "https://newsapi.org/v2/top-headlines"
    headers = {"Authorization": f"Bearer {api_key}"}

    for domain in domains:
        print(f"Récupération des articles pour la catégorie : {domain}")
        params = {"category": domain, "language": "en", "pageSize": 30}  # Limiter à 30 articles par catégorie
        response = requests.get(base_url, headers=headers, params=params)

        if response.status_code != 200:
            print(f"Échec lors de la récupération des articles pour {domain} : {response.status_code}")
            continue

        articles = response.json().get("articles", [])
        domain_path = os.path.join(save_path, domain)
        os.makedirs(domain_path, exist_ok=True)  # Créer le répertoire de la catégorie si nécessaire

        for article in articles:
            url = article.get("url")
            title = article.get("title", "Sans titre")
            author = article.get("author", "Auteur inconnu")

            if not url:
                continue  # Passer les articles sans URL valide

            try:
                print(f"Traitement de l'URL : {url}")
                if url != "https://removed.com":  # Filtrer les URLs non valides
                    article_text = extract_text_from_url(url)
                    if article_text:
                        # Enregistrer l'article dans un fichier texte
                        file_path = save_article(domain_path, article_text, title)

                        # Ajouter les métadonnées au fichier JSON
                        metadata.append({
                            "id": article_id,
                            "title": title,
                            "author": author,
                            "type": "article d'actualité",
                            "categorie": domain,
                            "file_path": file_path,
                            "url": url
                        })
                        article_id += 1
            except Exception as e:
                print(f"Erreur lors du traitement de {url} : {e}")

    # Sauvegarder les métadonnées dans le fichier JSON
    with open(metadata_file, "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)
    print(f"Les métadonnées ont été enregistrées dans {metadata_file}")

def extract_text_from_url(url):
    """
    Extrait le contenu textuel d'une URL donnée en utilisant la bibliothèque Newspaper.

    Args:
        url (str): URL de l'article.

    Returns:
        str: Contenu textuel extrait de l'article.
    """
    article = Article(url)
    article.download()
    article.parse()
    content = article.text  # Extraire le contenu principal
    return content

def save_article(path, text, title):
    """
    Sauvegarde le contenu de l'article dans un fichier texte.

    Args:
        path (str): Répertoire pour sauvegarder l'article.
        text (str): Contenu textuel de l'article.
        title (str): Titre de l'article (utilisé pour nommer le fichier).

    Returns:
        str: Chemin complet du fichier sauvegardé.
    """
    safe_title = "".join(c if c.isalnum() else "_" for c in title)[:50]  # Nettoyer le titre pour un nom de fichier valide
    file_path = os.path.join(path, f"{safe_title}.txt")
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)
    print(f"Article sauvegardé dans {file_path}")
    return file_path

# Application principale
if __name__ == "__main__":
    API_KEY = "35b99c0ae87b421390320689dee7fda3"  # Remplacez par votre propre clé API

    # Liste des catégories supportées par l'API
    DOMAINS = [
        "business", "entertainment", "general", "health", "science", "sports", "technology"
    ]

    fetch_articles(API_KEY, DOMAINS)


Récupération des articles pour la catégorie : business
Traitement de l'URL : https://www.washingtonpost.com/business/2024/12/27/2025-economic-outlook-trump-policies/
Article sauvegardé dans corpus/articles_actualite\business\The_2025_economy__5_things_to_watch___The_Washingt.txt
Traitement de l'URL : https://techcrunch.com/2024/12/27/openai-lays-out-its-for-profit-transition-plans/
Article sauvegardé dans corpus/articles_actualite\business\OpenAI_lays_out_its_for_profit_transition_plans___.txt
Traitement de l'URL : https://www.cnbc.com/2024/12/26/stock-market-today-live-update.html
Article sauvegardé dans corpus/articles_actualite\business\Dow_falls_more_than_200_points__but_on_track_to_po.txt
Traitement de l'URL : https://finance.yahoo.com/news/bank-america-buy-990-million-142437359.html
Article sauvegardé dans corpus/articles_actualite\business\Bank_of_America_to_Buy__990_Million_of_Multifamily.txt
Traitement de l'URL : https://removed.com
Traitement de l'URL : https://www.bloomberg.

# Collection de documents (articles d'actualite)

In [6]:
import os 
import json
import re
from newspaper import Article
import feedparser

def save_article(keyword_path, content, title):
    """
    Sauvegarde le contenu d'un article dans un fichier texte.

    Args:
        keyword_path (str): Répertoire où sauvegarder l'article.
        content (str): Contenu complet de l'article.
        title (str): Titre de l'article.

    Returns:
        str: Chemin complet du fichier enregistré.
    """
    # Nettoyage du titre pour éviter les caractères non valides dans les noms de fichiers
    safe_title = re.sub(r'[^\w\s]', '', title).replace(" ", "_")
    file_name = f"{safe_title}.txt"
    file_path = os.path.join(keyword_path, file_name)

    # Écrire le contenu complet dans un fichier texte
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content)

    return file_path

def fetch_article_with_newspaper(article_url):
    """
    Récupère le contenu complet d'un article à partir de son URL en utilisant Newspaper.

    Args:
        article_url (str): URL de l'article.

    Returns:
        str: Contenu complet de l'article ou None si une erreur survient.
    """
    try:
        # Utilisation de newspaper3k pour extraire le contenu de l'article
        article = Article(article_url)
        article.download()
        article.parse()
        return article.text, article.authors
    except Exception as e:
        print(f"Erreur avec Newspaper : {e}")
        return None, None

def fetch_scientific_articles(save_path="corpus/articles_actualite", metadata_file="all_documents.json"):
    """
    Récupère des articles de presse à partir de flux RSS, extrait leur contenu complet, et enregistre les métadonnées.

    Args:
        save_path (str): Répertoire où sauvegarder les articles extraits.
        metadata_file (str): Fichier JSON pour sauvegarder les métadonnées des articles extraits.
    """
    # Charger ou initialiser les métadonnées
    if os.path.exists(metadata_file):
        with open(metadata_file, "r", encoding="utf-8") as f:
            metadata = json.load(f)
    else:
        metadata = []

    article_id = len(metadata) + 1  # ID unique pour chaque article

    # URLs des flux RSS et leurs catégories correspondantes
    feed_urls = [
        "https://rss.nytimes.com/services/xml/rss/nyt/World.xml",
        "https://rss.nytimes.com/services/xml/rss/nyt/US.xml",
        "https://rss.nytimes.com/services/xml/rss/nyt/Politics.xml",
        "https://rss.nytimes.com/services/xml/rss/nyt/Business.xml",
        "https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml",
        "https://rss.nytimes.com/services/xml/rss/nyt/Sports.xml",
        "https://rss.nytimes.com/services/xml/rss/nyt/Opinion.xml"
    ]
    categories = ["world", "u.s", "politics", "business", "technology", "sports", "opinion"]

    for i, feed_url in enumerate(feed_urls):
        # Récupérer et analyser le flux RSS
        feed = feedparser.parse(feed_url)

        # Parcourir chaque article du flux RSS
        for entry in feed.entries:
            title = entry.title
            summary = entry.summary  # Utilisation de 'summary' ou 'content' si disponible
            url = entry.link

            if not url:
                print(f"Article {title} ignoré : URL manquant.")
                continue

            try:
                # Récupérer le contenu complet de l'article
                full_content, authors = fetch_article_with_newspaper(url)
                if not full_content:
                    full_content = summary  # Si le contenu complet est introuvable, utiliser le résumé

                # Sauvegarder l'article
                keyword = categories[i]  # Catégorie correspondant au flux RSS
                keyword_path = os.path.join(save_path, keyword)
                os.makedirs(keyword_path, exist_ok=True)

                file_path = save_article(keyword_path, full_content, title)

                # Ajouter les métadonnées
                metadata.append({
                    "id": article_id,
                    "title": title,
                    "author": authors if authors else "Auteur inconnu",
                    "type": "article d'actualites",
                    "categorie": keyword,
                    "file_path": file_path,
                    "url": url
                })
                article_id += 1
            except Exception as e:
                print(f"Erreur lors de l'enregistrement de {title} : {e}")

    # Sauvegarder les métadonnées dans un fichier JSON
    with open(metadata_file, "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)
    print(f"Les métadonnées des articles ont été enregistrées dans {metadata_file}")

# Application
if __name__ == "__main__":
    fetch_scientific_articles()


Erreur avec Newspaper : Article `download()` failed with HTTPSConnectionPool(host='www.nytimes.com', port=443): Read timed out. on URL https://www.nytimes.com/2024/12/28/opinion/joe-biden-presidency.html
Erreur avec Newspaper : Article `download()` failed with HTTPSConnectionPool(host='www.nytimes.com', port=443): Read timed out. on URL https://www.nytimes.com/2024/12/23/opinion/democrats-election-future.html
Les métadonnées des articles ont été enregistrées dans all_documents.json



# Preprocessing des docs

In [7]:
import os
import json
import collections
from transformers import AutoTokenizer

# Charger le tokenizer de DistilBERT
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def read_documents(json_file_path):
    """
    Lit le fichier JSON, extrait les ids et les chemins des fichiers, lit le contenu de chaque fichier,
    et retourne un dictionnaire associant chaque id à son contenu.

    :param json_file_path: Chemin du fichier JSON contenant les métadonnées des documents (id, chemin du fichier).
    :return: Un dictionnaire {id: contenu du document}.
    """
    try:
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            documents = json.load(json_file)
        
        doc_contents = {}
        for doc in documents:
            doc_id = doc.get("id")
            file_path = doc.get("file_path")
            # Vérification de l'existence du fichier et des informations nécessaires
            if doc_id and file_path and os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                doc_contents[doc_id] = content
            else:
                print(f"Fichier introuvable ou informations manquantes pour l'id {doc_id}: {file_path}")
        return doc_contents

    except Exception as e:
        print(f"Une erreur s'est produite lors de la lecture des documents : {e}")
        return {}

def preprocess_documents(documents_content):
    """
    Applique le prétraitement aux documents :
    - Tokenisation (utilisation de DistilBERT)
    - Construction du vocabulaire global à partir des tokens

    :param documents_content: Dictionnaire {id: contenu brut des documents}.
    :return: Dictionnaire avec le contenu prétraité et le vocabulaire global sous forme de Counter.
    """
    vocabulary = collections.Counter()
    preprocessed_content = {}

    for doc_id, content in documents_content.items():
        # Tokenisation du contenu du document avec gestion de la longueur des séquences
        tokens = tokenizer(content, truncation=True, padding=True, max_length=512, return_tensors="pt")
        
        # Enregistrer les tokens sous forme de dictionnaire avec input_ids et attention_mask
        preprocessed_content[doc_id] = {
            'input_ids': tokens['input_ids'].squeeze().tolist(),  # Convertir en liste simple
            'attention_mask': tokens['attention_mask'].squeeze().tolist()  # Convertir en liste simple
        }
        
        # Mise à jour du vocabulaire global
        vocabulary.update(tokens['input_ids'].squeeze().tolist())

    return preprocessed_content, vocabulary

def save_preprocessed_data(preprocessed_content, vocabulary, output_file="preprocessed_content.json"):
    """
    Sauvegarde le contenu prétraité et le vocabulaire dans un fichier JSON.

    :param preprocessed_content: Dictionnaire {id: contenu prétraité} (tokens).
    :param vocabulary: Vocabulaire global sous forme de `collections.Counter` (liste des ids).
    :param output_file: Chemin du fichier de sortie pour enregistrer les données.
    """
    data_to_save = {
        "preprocessed_content": preprocessed_content,
        "vocabulary": list(vocabulary.keys())  # Utiliser les clés du Counter comme vocabulaire
    }

    try:
        with open(output_file, 'w', encoding='utf-8') as json_file:
            json.dump(data_to_save, json_file, indent=4, ensure_ascii=False)
        print(f"Les contenus prétraités et le vocabulaire ont été enregistrés dans le fichier : {output_file}")
    except Exception as e:
        print(f"Une erreur s'est produite lors de la sauvegarde des données : {e}")

# Application principale
if __name__ == "__main__":
    json_file_path = "all_documents.json"
    documents_content = read_documents(json_file_path)

    if documents_content:
        preprocessed_content, vocabulary = preprocess_documents(documents_content)
        save_preprocessed_data(preprocessed_content, vocabulary)


c:\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Les contenus prétraités et le vocabulaire ont été enregistrés dans le fichier : preprocessed_content.json


# Representation des documents sous forme de vecteur

In [10]:
from transformers import AutoTokenizer, AutoModel
import torch
import json

# Charger le modèle et le tokenizer DistilBERT
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

def load_preprocessed_content(input_file="preprocessed_content.json"):
    """
    Charge le contenu prétraité et le vocabulaire depuis un fichier JSON.

    Args:
        input_file (str): Nom du fichier JSON contenant les données prétraitées et le vocabulaire.

    Returns:
        tuple: Un tuple (preprocessed_content, vocabulary), où `preprocessed_content` est un dictionnaire
               {id_document: contenu prétraité (tokens)} et `vocabulary` est une liste des termes.
    """
    try:
        with open(input_file, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)

        preprocessed_content = data.get("preprocessed_content", {})
        vocabulary = data.get("vocabulary", [])
        return preprocessed_content, vocabulary
    except FileNotFoundError:
        print(f"Erreur : Le fichier {input_file} n'a pas été trouvé.")
        return {}, []
    except json.JSONDecodeError:
        print(f"Erreur : Le fichier {input_file} n'est pas un fichier JSON valide.")
        return {}, []
    except Exception as e:
        print(f"Une erreur s'est produite lors de la lecture du fichier : {e}")
        return {}, []

def create_distilbert_embeddings(doc_contents, output_file="distilbert_vectors.json"):
    """
    Calcule les embeddings pour chaque document en utilisant DistilBERT et une méthode de pooling attentionné.

    Args:
        doc_contents (dict): Dictionnaire {id_document: contenu prétraité (tokens)}.
        output_file (str): Nom du fichier JSON pour enregistrer les embeddings.

    Returns:
        None
    """
    try:
        if not doc_contents:
            print("Erreur : Aucun contenu de document fourni pour calculer les embeddings.")
            return

        document_vectors = {}
        
        for doc_id, tokens in doc_contents.items():
            # Assurez-vous que `tokens` est un dictionnaire contenant 'input_ids' et 'attention_mask'
            input_ids = torch.tensor(tokens['input_ids'], dtype=torch.long)  # Conversion explicite en tenseur
            attention_mask = torch.tensor(tokens['attention_mask'], dtype=torch.long)

            # Ajout d'une dimension batch
            input_ids = input_ids.unsqueeze(0)  # [batch_size, seq_len]
            attention_mask = attention_mask.unsqueeze(0)  # [batch_size, seq_len]

            # Calcul des embeddings avec le modèle DistilBERT
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                embeddings = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]

            # Attention pooling
            attention_weights = attention_mask.unsqueeze(-1)  # [batch_size, seq_len, 1]
            pooled_embeddings = torch.sum(embeddings * attention_weights, dim=1) / attention_weights.sum(dim=1)

            # Ajout des embeddings au dictionnaire
            document_vectors[doc_id] = pooled_embeddings.squeeze(0).tolist()


        # Sauvegarder les embeddings dans un fichier JSON
        with open(output_file, 'w', encoding='utf-8') as json_file:
            json.dump(document_vectors, json_file, indent=4)

        print(f"Les vecteurs DistilBERT des documents ont été enregistrés dans le fichier {output_file}.")
    except Exception as e:
        print(f"Une erreur s'est produite lors du calcul des embeddings : {e}")

# Application principale
if __name__ == "__main__":
    # Charger les contenus prétraités
    preprocessed_content, vocabulary = load_preprocessed_content()

    # Vérifier si des contenus ont été chargés avec succès
    if preprocessed_content:
        # Créer les vecteurs avec DistilBERT
        create_distilbert_embeddings(preprocessed_content, output_file="distilbert_vectors.json")
    else:
        print("Impossible de générer les vecteurs car aucun contenu prétraité n'a été chargé.")


Les vecteurs DistilBERT des documents ont été enregistrés dans le fichier distilbert_vectors.json.


# Le calcule de similarites

In [12]:
import csv
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json

def compute_similarity_matrix(vector_file="distilbert_vectors.json", output_file="document_similarity_matrix.csv"):
    """
    Calcule la similarité cosinus entre tous les documents et enregistre les résultats dans un fichier CSV sous forme de matrice.

    :param vector_file: Nom du fichier JSON contenant les vecteurs des documents.
    :param output_file: Nom du fichier CSV pour enregistrer la matrice des similarités.
    """
    try:
        # Charger les vecteurs depuis le fichier JSON
        with open(vector_file, 'r', encoding='utf-8') as json_file:
            document_vectors = json.load(json_file)
        
        # Extraire les IDs et les vecteurs
        doc_ids = list(document_vectors.keys())
        vectors = np.array([document_vectors[doc_id] for doc_id in doc_ids])
        
        # Calculer la matrice de similarité cosinus
        similarity_matrix = cosine_similarity(vectors)
        
        # Enregistrer la matrice dans un fichier CSV
        with open(output_file, 'w', encoding='utf-8', newline='') as csv_file:
            writer = csv.writer(csv_file)
            
            # Écrire l'en-tête (IDs des documents)
            writer.writerow([""] + doc_ids)
            
            # Écrire chaque ligne (ID + similarités)
            for i, doc_id in enumerate(doc_ids):
                writer.writerow([doc_id] + similarity_matrix[i].tolist())
        
        print(f"La matrice de similarité a été enregistrée dans le fichier {output_file}.")
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")


def save_sorted_similarities_from_matrix(matrix_file="document_similarity_matrix.csv", output_file="sorted_document_similarities.csv"):
    """
    Enregistre dans un fichier CSV les IDs des documents les plus similaires pour chaque document,
    triés par similarité décroissante, à partir d'une matrice de similarité déjà calculée.

    :param matrix_file: Nom du fichier CSV contenant la matrice de similarité.
    :param output_file: Nom du fichier CSV pour enregistrer les similarités triées (seulement les IDs).
    """
    try:
        # Charger la matrice de similarité depuis le fichier CSV
        with open(matrix_file, 'r', encoding='utf-8') as csv_file:
            reader = csv.reader(csv_file)
            rows = list(reader)
        
        # Extraire les IDs des documents (en-tête)
        doc_ids = rows[0][1:]
        
        # Préparer les similarités triées (seulement les IDs)
        sorted_similarities = []
        for i, row in enumerate(rows[1:]):
            doc_id = row[0]
            similarities = [(doc_ids[j], float(row[j + 1])) for j in range(len(doc_ids)) if i != j]
            
            # Trier les similarités par ordre décroissant et récupérer uniquement les IDs
            sorted_doc_ids = [sim_doc_id for sim_doc_id, _ in sorted(similarities, key=lambda x: x[1], reverse=True)]
            sorted_similarities.append((doc_id, sorted_doc_ids))
        
        # Enregistrer les IDs triés dans un fichier CSV
        with open(output_file, 'w', encoding='utf-8', newline='') as csv_file:
            writer = csv.writer(csv_file)
            
            # Écrire l'en-tête (Document + Liste d'IDs des documents similaires)
            header = ["Document"] + [f"Similar Doc {i+1}" for i in range(len(doc_ids) - 1)]
            writer.writerow(header)
            
            # Écrire les similarités triées pour chaque document
            for doc_id, sorted_doc_ids in sorted_similarities:
                # Compléter avec des colonnes vides si moins de documents similaires
                row = [doc_id] + sorted_doc_ids + [''] * (len(doc_ids) - 1 - len(sorted_doc_ids))
                writer.writerow(row)
        
        print(f"Les similarités triées ont été enregistrées dans le fichier {output_file}.")
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

# Application
if __name__ == "__main__":
    compute_similarity_matrix(vector_file="distilbert_vectors.json", output_file="document_similarity_matrix.csv")
    save_sorted_similarities_from_matrix(
        matrix_file="document_similarity_matrix.csv",
        output_file="sorted_document_similarities.csv"
    )


La matrice de similarité a été enregistrée dans le fichier document_similarity_matrix.csv.
Les similarités triées ont été enregistrées dans le fichier sorted_document_similarities.csv.
